In [ ]:
%pip install xhtml2pdf

In [59]:
from IPython.display import display, HTML
from xhtml2pdf import pisa 
from aurora_postgres_helpers import SQLConnection
from datetime import datetime

### instantiate SQLConnection class

In [55]:
s = SQLConnection()

### Quering database

In [61]:
def get_current_date() -> str:
    current_date = str(datetime.now().date())
    return current_date
current_date = get_current_date()

In [ ]:
def filter_by_date(s:SQLConnection, date)

#### Number of rides completed in the past day

In [58]:

number_of_rides = s.read_query(query = "SELECT * FROM yusra_stories_production.rides ")
number_of_rides

,ride_id,user_id,start_time,end_time,total_duration,max_heart_rate_bpm,min_heart_rate_bpm,avg_heart_rate_bpm,avg_resistance,avg_rpm,total_power_kilojoules
0,1,4651,2022-10-07 15:08:31,2022-10-07 15:17:13,0:08:40,157,71,107,41,47,18.01
1,2,4651,2022-10-07 15:17:14,2022-10-07 15:25:55,0:08:40,151,85,105,41,48,12.75
2,3,4651,2022-10-07 15:25:57,2022-10-07 15:34:38,0:08:40,155,80,121,41,50,13.93
3,5,4653,2022-10-07 15:43:22,2022-10-07 15:49:17,0:05:54,168,81,133,44,44,14.56
4,4,4653,2022-10-07 15:34:40,2022-10-07 15:43:21,0:08:40,158,80,111,40,41,7.29
5,6,4653,2022-10-07 15:49:19,2022-10-07 15:58:00,0:08:40,157,67,114,39,41,8.53


#### Gender split of riders of the past day

In [ ]:
query = """
    
"""
riders_gender_split = s.read_query()

#### Ages of the riders of the past day

#### Average power and heart rate of riders of past day

### Create the HTML Template

In [32]:
def report_block_template( graph_relative_path, caption=''):

    graph_block = (''
            '<a href="{graph_relative_path}" target="_blank">' # Open the interactive graph when you click on the image
                '<img style="height: 400px;" img src="{graph_relative_path}">'
            '</a>')

    report_block = ('' +
        graph_block +
        '{caption}' + 
        '<br>'      + 
        '</a>' +
        '<br>' +
        '<hr>')                       
    report_layout = (
       '<h2>Deloton Exercise Bikes Daily Report</h2>'
       + '<hr>'
       + report_block
    )
    return report_layout.format(graph_relative_path=graph_relative_path, caption=caption)


graph_relative_path = 'age_distr.png'

report = report_block_template(graph_relative_path, caption='Dis a graph')


In [33]:
display(HTML(report))

### Convert the HTML to PDF

In [28]:

def convert_html_to_pdf(source_html, output_filename):

    result_file = open(output_filename, "w+b")

    pisa_status = pisa.CreatePDF(
            source_html,           
            dest=result_file)           

    result_file.close()           

    return pisa_status.err

In [29]:
convert_html_to_pdf(report, 'report.pdf')

0

In [30]:
! open report.pdf